In [53]:
import sqlite3
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import numpy as np
from geopy.distance import distance
import re
# from  keys  import  client_id, api_key
import folium

In [54]:
pd.set_option('display.max_rows', 200)

In [55]:
from cleaning_utils import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
blight = pd.read_csv('../data/Blight_Violations.csv')

/Users/kelvinarellano/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (9,11,15,16,17,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [57]:
blight.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496335 entries, 0 to 496334
Data columns (total 36 columns):
X                             306227 non-null float64
Y                             306227 non-null float64
ticket_id                     496335 non-null int64
ticket_number                 496335 non-null object
agency_name                   496335 non-null object
inspector_name                496335 non-null object
violator_name                 496333 non-null object
violation_street_number       496335 non-null int64
violation_street_name         496270 non-null object
violation_zip_code            154162 non-null object
violator_id                   496335 non-null int64
mailing_address_str_number    496329 non-null object
mailing_address_str_name      496325 non-null object
city                          493079 non-null object
state                         492616 non-null object
zip_code                      493076 non-null object
non_us_str_code               3259 non-nul

In [58]:
MCM = pd.read_csv('../data/Motor_City_Mapping,_Winter_2013-14_Certified_Results.csv')

/Users/kelvinarellano/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [59]:
MCM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 379549 entries, 0 to 379548
Data columns (total 33 columns):
OBJECTID             379549 non-null int64
D3_SurveyParcelID    379549 non-null object
CityParcelID2013     379549 non-null object
AddCombo             379549 non-null object
Address_Number       379549 non-null object
Address_Direction    379549 non-null object
Street               379549 non-null object
Zip                  379549 non-null object
PCount               379549 non-null int64
Blext_ID             377209 non-null float64
User_ID              373229 non-null float64
Reviewer_ID          359568 non-null float64
Time_Surveyed        379549 non-null object
Updated_At           379549 non-null object
Status               373229 non-null float64
Photo_URL            373217 non-null object
Structure            373227 non-null object
Use_                 373223 non-null object
Condition            261355 non-null object
Occupancy            261202 non-null object
Needs_B

In [60]:
MCM['CityParcelID2013'] = [x[:7] for x in MCM.CityParcelID2013]
MCM['D3_SurveyParcelID'] = [x[:7] for x in MCM.D3_SurveyParcelID]

In [61]:

MCM.drop(['GeoID10_BG','NAME10_Block','GEOID10_Block','Notes','Photo_URL','Time_Surveyed','Reviewer_ID','User_ID','Blext_ID','PCount','OBJECTID'], axis=1, inplace = True)

In [62]:
MCM.drop(['GEOID10_Tract','Updated_At'], axis=1, inplace = True)

In [63]:
blight = blight[blight.payment_status == 'PAID IN FULL']

In [64]:
columns_to_drop = [
                 
                 'ticket_number',
                 'agency_name',
                 'inspector_name',
                 'violator_name',
                
                 'violator_id',
                 'mailing_address_str_number',
                 'mailing_address_str_name',
                 'city',
                 'state',
                 'zip_code',
                 'non_us_str_code',
                 'country',
                 
                 'ticket_issued_time',
                 'hearing_date',
                 'hearing_time',
                 
                 'violation_code',
                 'violation_description',
                 'disposition',
                 'clean_up_cost',
                 'payment_amount',
                 
                 
                 'collection_status',
                 'violation_address',
                 
                 'oid',
                 'geom',
                 'fine_amount']

In [65]:
blight = blight.drop(columns=columns_to_drop)

In [66]:
blight.violation_date = pd.to_datetime(blight['violation_date']).dt.date
blight.judgment_date = pd.to_datetime(blight['judgment_date']).dt.date
blight.payment_date = pd.to_datetime(blight['payment_date']).dt.date

In [67]:
blight.rename({'Y':'lat', 'X':'lon', 'parcelno': 'parcel_id'}, axis=1, inplace=True)

In [68]:
blight['compliance'] = blight.apply(compliance, axis=1)

In [69]:
blight = blight.dropna(subset=['parcel_id'])

In [39]:
#blight['parcel_id'] = blight.apply(parse_parcel,axis=1)


In [70]:
blight['coordinates'] = blight.apply(coord_bligth,axis=1)

In [71]:
crimes = pd.read_csv('../data/Reported_Major_Crimes_2011_to_2014.csv')

In [72]:
crimes['lat'] = crimes.LOCATION.map(extract_lat_crime)
crimes['lon'] = crimes.LOCATION.map(extract_long_crime)

In [73]:
crimes =  crimes[ (crimes['lat'] < 45) | (crimes['lon'] < 85) ]

In [74]:
crimes['coordinates'] = crimes.apply(coord_bligth, axis=1)

In [75]:
crimes.INCIDENTDATE = pd.to_datetime(crimes['INCIDENTDATE']).dt.date

In [76]:
crimes.head(150)

,ROWNUM,CATEGORY,OFFENSEDESCRIPTION,STATEOFFENSEFILECLASS,INCIDENTDATE,HOUR,SCA,PRECINCT,NEIGHBORHOOD,CENSUSTRACT,LOCATION,ObjectId,lat,lon,coordinates
0,1311873,STOLEN VEHICLE,VEHICLE THEFT,24001,2011-01-02,0,1103.0,11.0,PERSHING,5066.0,"00 EUREKA/E LANTZ\n(42.4382, -83.0579)",1,42.4382,-83.0579,"(42.4382, -83.0579)"
1,1311874,AGGRAVATED ASSAULT,AGG/FEL ASSAULT - NON-FAMILY - OTHER WEAPON,13002,2011-01-02,11,709.0,7.0,LAFAYETTE PARK,5166.0,"E.LAFAYETTE01800\n(42.3394, -83.0295)",2,42.3394,-83.0295,"(42.3394, -83.0295)"
2,1311878,LARCENY,"LARCENY - FROM BUILDING (INCLUDES LIBRARY, OFF...",23003,2011-01-02,11,1105.0,11.0,OUTER DRIVE VAN DYKE,5051.0,"VANDYKE19900\n(42.4415, -83.0238)",3,42.4415,-83.0238,"(42.4415, -83.0238)"
3,1311879,LARCENY,LARCENY (OTHER),23007,2011-01-01,20,706.0,7.0,CHENE,5188.0,"JOS CAMPAU04100\n(42.3606, -83.0339)",4,42.3606,-83.0339,"(42.3606, -83.0339)"
4,1311883,STOLEN VEHICLE,VEHICLE THEFT,24001,2011-01-02,3,612.0,6.0,WARRENDALE,5457.0,"PIEDMONT06000\n(42.3328, -83.228)",5,42.3328,-83.2280,"(42.3328, -83.228)"
5,1311884,BURGLARY,BURGLARY - BURGLARY - FORCED ENTRY - RESIDENCE,22001,2011-01-02,12,1101.0,11.0,STATE FAIR-NOLAN,NaN,"ANDOVER19300\n(42.4349, -83.0996)",6,42.4349,-83.0996,"(42.4349, -83.0996)"
6,1311885,LARCENY,LARCENY - PARTS AND ACCESSORIES FROM VEHICLE,23006,2011-01-02,11,709.0,7.0,LAFAYETTE PARK,5170.0,"JAY01800\n(42.3453, -83.0352)",7,42.3453,-83.0352,"(42.3453, -83.0352)"
7,1311886,LARCENY,LARCENY - PERSONAL PROPERTY FROM VEHICLE,23005,2011-01-02,0,810.0,8.0,RIVERDALE,5432.0,"LAHSER16600\n(42.4106, -83.2571)",8,42.4106,-83.2571,"(42.4106, -83.2571)"
8,1311898,LARCENY,LARCENY - PARTS AND ACCESSORIES FROM VEHICLE,23006,2011-01-02,10,1112.0,11.0,CITY AIRPORT,5048.0,"DOYLE08000\n(42.4169, -83.023)",9,42.4169,-83.0230,"(42.4169, -83.023)"
9,1311899,AGGRAVATED ASSAULT,AGG/FEL ASSAULT - NON-FAMILY - GUN,13002,2011-01-02,13,611.0,6.0,WACO,5455.0,"SOUTHFIELD07200\n(42.344, -83.2154)",10,42.3440,-83.2154,"(42.344, -83.2154)"


In [79]:
MCM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 379549 entries, 0 to 379548
Data columns (total 20 columns):
D3_SurveyParcelID    379549 non-null object
CityParcelID2013     379549 non-null object
AddCombo             379549 non-null object
Address_Number       379549 non-null object
Address_Direction    379549 non-null object
Street               379549 non-null object
Zip                  379549 non-null object
Status               373229 non-null float64
Structure            373227 non-null object
Use_                 373223 non-null object
Condition            261355 non-null object
Occupancy            261202 non-null object
Needs_Boarding       261271 non-null object
Fire                 261284 non-null object
Dumping              373169 non-null object
Units                245276 non-null object
Improved             112058 non-null object
Maintained           112005 non-null object
PublicUse            7351 non-null object
NAMELSAD10_Tract     379549 non-null object
dtypes: fl

In [80]:
blight.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59904 entries, 7 to 494929
Data columns (total 13 columns):
lon                        59904 non-null float64
lat                        59904 non-null float64
ticket_id                  59904 non-null int64
violation_street_number    59904 non-null int64
violation_street_name      59902 non-null object
violation_zip_code         22995 non-null object
violation_date             59904 non-null object
judgment_date              59904 non-null object
payment_date               59904 non-null object
payment_status             59904 non-null object
parcel_id                  59904 non-null object
compliance                 59904 non-null int64
coordinates                59904 non-null object
dtypes: float64(2), int64(3), object(8)
memory usage: 6.4+ MB


In [83]:
blight['parcel_id'] = [x[:7] for x in blight.parcel_id]


In [85]:
blight.head(150)

,lon,lat,ticket_id,violation_street_number,violation_street_name,violation_zip_code,violation_date,judgment_date,payment_date,payment_status,parcel_id,compliance,coordinates
7,-83.019172,42.339196,18653,2900,JEFFERSON,NaN,2005-01-14,2005-02-23,2005-02-22,PAID IN FULL,1100008,1,"(42.339195641000096, -83.01917209300001)"
11,-82.942097,42.388525,18657,15439,MACK,NaN,2005-01-14,2005-02-18,2005-03-09,PAID IN FULL,2100142,0,"(42.3885247860001, -82.942097334)"
12,-82.946176,42.371778,18658,14214,JEFFERSON,NaN,2005-01-14,2005-02-04,2005-02-04,PAID IN FULL,2100035,1,"(42.371778185, -82.946176343)"
15,-83.045750,42.335121,18661,1310,BROADWAY,NaN,2005-01-14,2005-02-18,2007-06-12,PAID IN FULL,0100400,0,"(42.335120859, -83.0457496719999)"
18,-83.044467,42.335210,18664,343,MACOMB,NaN,2005-01-14,2005-03-23,2007-05-31,PAID IN FULL,0100020,0,"(42.335210488, -83.0444668339999)"
19,-83.047308,42.335973,18665,1400,BROADWAY,NaN,2005-01-14,2005-02-18,2008-03-07,PAID IN FULL,0100401,0,"(42.3359729110001, -83.04730827)"
20,-83.043959,42.335840,18666,426,CLINTON,NaN,2005-01-14,2005-02-18,2009-12-14,PAID IN FULL,0100021,0,"(42.335840128, -83.04395936)"
44,-83.081225,42.443874,18691,20200,CONANT,NaN,2005-01-18,2005-02-21,2005-02-21,PAID IN FULL,1302530,1,"(42.443874183, -83.081224875)"
48,-83.008388,42.344630,18695,6431,JEFFERSON,NaN,2005-01-18,2005-01-27,2005-01-27,PAID IN FULL,1500003,1,"(42.34462999, -83.00838816899991)"
58,-83.098182,42.319529,18705,4650,VERNOR,NaN,2005-01-18,2016-11-22,2016-11-22,PAID IN FULL,1600079,1,"(42.3195285400001, -83.098182026)"


In [84]:
blight.merge(MCM, how='inner', left_on = 'D3_SurveyParcelID',right_on = 'parcel_id')

KeyError: 'parcel_id'